<a href="https://colab.research.google.com/github/CynYZhu/MIDS_266_FinalProject/blob/main/266_Project_Goding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# W266 Project

In [1]:
#install tensorflow datasets library
!pip install datasets
!pip install huggingface-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from datasets import load_dataset
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from google.colab import files
import re

In [3]:
dataset_train, dataset_validation, dataset_test = load_dataset('trivia_qa', 'rc',split = ['train', 'validation', 'test'])

Reusing dataset trivia_qa (/root/.cache/huggingface/datasets/trivia_qa/rc/1.2.0/e73c5e47a8704744fa9ded33504b35a6c098661813d1c2a09892eb9b9e9d59ae)


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
from google.colab import drive
drive.mount('/content/drive')
!ls drive/MyDrive/w266/data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
dataset_test  dataset_train  dataset_validation


In [14]:
# match squad 2 annotations
dataset_train.save_to_disk("drive/MyDrive/w266/data/dataset_train.json")

In [18]:
#load dataset
!cd drive/MyDrive/w266/data/
!ls -lh drive/MyDrive/w266/data/
path = '/drive/MyDrive/w266/data/'


total 16K
drwx------ 2 root root 4.0K Jul 17 02:50 dataset_test
drwx------ 2 root root 4.0K Jul 17 02:46 dataset_train
drwx------ 2 root root 4.0K Jul 20 17:00 dataset_train.json
drwx------ 2 root root 4.0K Jul 17 02:49 dataset_validation


In [ ]:
train_set = load_dataset('json', data_files='drive/MyDrive/w266/data/dataset_train.json/dataset.arrow')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

ERROR:datasets.packaged_modules.json.json:Failed to read file '/content/drive/MyDrive/w266/data/dataset_train.json/dataset.arrow' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Invalid value. in row 0


In [ ]:
dataset_test[0].keys()

dict_keys(['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'])

In [ ]:
dataset_validation[0].keys()

dict_keys(['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'])

In [10]:
dataset_train[0].keys()

dict_keys(['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'])

In [ ]:
#check how many empty search_context values
counter = 0
for item in dataset_train:
  if item['search_results']['search_context'] != []:
    continue   
  else:
    counter +=1 

In [ ]:
print('There are ' + str(counter) + ' empty search_context in the dataset_train')

There are 65038 empty search_context in the dataset_train


In [ ]:
#total length of dataset_train
len(dataset_train)

138384

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls drive/MyDrive/w266/data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
dataset_test  dataset_train  dataset_validation


In [4]:
def clean_text(text):
  text = text.lower()
  text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ')
  return text

In [5]:
#create answer start function
def answer_start(dataset, context):
  for item in dataset['answer']['normalized_aliases']:
    answer_string = str(item)
    start_char = answer_string[0]
    start_index = context.find(start_char)
    return {'text': answer_string, 'answer_start': start_index}

In [11]:
#function to format train, validation, and test data
def format_dataset(dataset):

  #extract needed information
  question_id = dataset['question_id']
  question = dataset['question']
  #clean up question text, context by lower case, removing unicode chars, removing stop words
  question = clean_text(question)
  #wiki_context = dataset['entity_pages']['wiki_context']
  search_context = dataset['search_results']['search_context']
  #evaluate each item in search_context list and make one large context
  total_context = []
  for item in search_context:
    item = clean_text(item)
    total_context.append(item) 

  #normalized aliases
  answer = dataset['answer']['normalized_aliases']
 
  #join search context to one large string
  total_context_joined = ','.join(total_context)
  
  answer_text_list = []
  answer_start_list = []
  for item2 in answer:
    answer_string = str(item2)
    start_char = answer_string[0]
    start_index = total_context_joined.find(start_char)
    answer_text_list.append(answer_string)
    answer_start_list.append(start_index)


  #answer = answer_start(dataset, total_context_joined)

  return {'question_id': question_id,  'question': question, 'search_context': total_context_joined, 'answer': {'text': answer_text_list,'answer_start': answer_start_list}}


In [ ]:
#Scan first 76401 entries for search context fields, previous EDA showed majority of search context were located here
cleaned_traindata =[]
for x in range(0,76041):
    if dataset_train[x]['search_results']['search_context'] != []:
      cleaned_traindata.append(format_dataset(dataset_train[x]))
    else:
      continue

In [20]:
#clear list for RAM maint.
cleaned_traindata = []

In [27]:
#save first chucnk
with open('cleaned_traindata1.json', 'w') as f:
    json.dump(cleaned_traindata, f)

In [15]:
len(cleaned_traindata)

31560

In [18]:
cleaned_traindata[500]

{'answer': {'answer_start': [8, 1116, 18, 1116, 73, 8, 204],
  'text': ['homer bedloe',
   'j homer bedloe',
   'shady rest hotel',
   'j homer bedlow',
   'petticoat junction',
   'homer bedlow',
   'kate bradley']},
 'question': 'what was the first spin-off from the beverly hillbillies called?',
 'question_id': 'tc_923',
 'search_context': 'beverly hillbillies: season 3 beverly hillbillies: season 3 for the most part television sit-coms are a light and  fluffy form of family entertainment. during the seventies there were some  attempts to make cutting edge and socially relevant shows like �all in the  family� but the general rule of thumb stands that this type of tv programming is  silly fun. sometimes even a show like this can achieve a level of fame that  makes it a household name and the proverbial must see television. in 1962 one  such series got its start; �the beverly hillbillies�. it would become an instant  hit and result in several spin off series. at the time critics initia

In [20]:
#recreate cleaned trained data with nested answer info
def_finalformat(dataset):
  



['grace hegger',
 'lewis harry sinclair',
 'harry sinclair lewis',
 'sinclair lewis']

In [65]:
a = ['test', 'fun']
','.join(a)
a

['test', 'fun']

In [ ]:
a = ''